In [1]:
from data import *
from result import * 
from orion.data import load_signal, load_anomalies
import orion.evaluation as evaluate
import numpy as np
from joblib import Parallel, delayed
from gpt import *

In [2]:
#Load the signal and true anomalies
signal_name = 'E-1'
signal = load_signal(signal_name)
true_ano = load_anomalies(signal_name)

In [3]:
signal

,timestamp,value
0,1222819200,-1.0
1,1222840800,-1.0
2,1222862400,-1.0
3,1222884000,-1.0
4,1222905600,-1.0
...,...,...
11391,1468864800,-1.0
11392,1468886400,-1.0
11393,1468908000,-1.0
11394,1468929600,-1.0


In [4]:
#Get values and indices of signal 
values = signal['value'].values
indices = signal.index.values

In [5]:
#make rolling windows
window_size = 1000
step_size = 100

windows, start_indices = rolling_window_sequences(values, indices, window_size, step_size)

In [14]:
#get list of anomalous indices for each window
def get_anomalies(seq, num_iters = 1, msg_func = create_message_zero_shot, model_func = get_gpt_model_response):
    message = msg_func(sig2str(seq))
    res_list = []
    for i in range(num_iters): 
        res = model_func(message)
        ano_ind = str2ind(res, len(seq))
        res_list.append(ano_ind)
    return get_anomaly_list_within_seq(res_list)

In [18]:
final_ano = []
for seq in windows: 
    final_ano.append(get_anomalies(seq))

BadRequestError: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}